In [1]:
pip install pydantic


Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
API with JSON Validation using Pydantic - Complete Solution
Validate JSON input using Pydantic before processing
"""

from pydantic import BaseModel, Field, validator
from typing import Optional
import json

print("="*50)
print("PYDANTIC BASICS")
print("="*50)

class SimpleProduct(BaseModel):
    """A simple product model for validation."""
    name: str
    price: float
    quantity: int = 1  # Default value
    
    @validator('price')
    def price_must_be_positive(cls, v):
        """Validate that price is positive."""
        if v <= 0:
            raise ValueError('Price must be positive')
        return v
    
    @validator('quantity')
    def quantity_must_be_positive(cls, v):
        """Validate that quantity is positive."""
        if v <= 0:
            raise ValueError('Quantity must be positive')
        return v

# Test validation
print("\n1. Valid data:")
try:
    product1 = SimpleProduct(name="Widget", price=10.99, quantity=5)
    print(f"  ✓ Valid: {product1.name} - ${product1.price}")
except Exception as e:
    print(f"  ✗ Error: {e}")

print("\n2. Invalid data (negative price):")
try:
    product2 = SimpleProduct(name="Widget", price=-10.99)
except Exception as e:
    print(f"  ✗ Validation error (expected): {e}")

print("\n✓ Pydantic basics working!")


PYDANTIC BASICS

1. Valid data:
  ✓ Valid: Widget - $10.99

2. Invalid data (negative price):
  ✗ Validation error (expected): 1 validation error for SimpleProduct
price
  Value error, Price must be positive [type=value_error, input_value=-10.99, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

✓ Pydantic basics working!


/var/folders/xb/pfzh1bqs77z0b1bpr63lnfyw0000gn/T/ipykernel_11556/1097675553.py:20: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('price')
/var/folders/xb/pfzh1bqs77z0b1bpr63lnfyw0000gn/T/ipykernel_11556/1097675553.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('quantity')


In [2]:
"""
API with JSON Validation using Pydantic - Step 2 Updated
Creating Product Data Models with Pydantic V2 syntax
"""
from pydantic import BaseModel, Field, field_validator
from typing import Optional
from enum import Enum

# Define categories as an Enum (like a dropdown menu of valid options)
class ProductCategory(str, Enum):
    ELECTRONICS = "electronics"
    CLOTHING = "clothing"
    HOME_GOODS = "home_goods"
    BOOKS = "books"
    OTHER = "other"

class Product(BaseModel):
    """Main product model for validation."""
    name: str = Field(..., min_length=1, max_length=100, description="Product name")
    price: float = Field(..., gt=0, description="Price must be positive")
    category: ProductCategory = Field(..., description="Product category")
    description: Optional[str] = Field(None, max_length=500, description="Optional description")
    brand: Optional[str] = Field(None, max_length=50, description="Brand name")
    in_stock: bool = Field(True, description="Availability status")
    
    # Custom validator for price - Updated V2 syntax
    @field_validator('price')
    @classmethod
    def price_not_too_high(cls, v):
        """Validate that price is reasonable (under $1,000,000)."""
        if v > 1000000:
            raise ValueError('Price seems unrealistically high')
        return v
    
    # Custom validator for name - Updated V2 syntax
    @field_validator('name')
    @classmethod
    def name_must_contain_letters(cls, v):
        """Validate that name contains at least some letters."""
        if not any(c.isalpha() for c in v):
            raise ValueError('Name must contain letters')
        return v.strip()

class ProductListingRequest(BaseModel):
    """Full request for generating a product listing."""
    product: Product
    target_audience: str = Field("general", description="Target audience for the listing")
    tone: str = Field("professional", description="Tone of the listing")
    language: str = Field("English", description="Language for the listing")

print("="*50)
print("TESTING PRODUCT MODELS (Pydantic V2 Syntax)")
print("="*50)

# Test 1: Valid product
print("\n1. Testing valid product:")
try:
    valid_product = Product(
        name="iPhone 15 Pro",
        price=999.99,
        category=ProductCategory.ELECTRONICS,
        description="Latest Apple smartphone with advanced camera",
        brand="Apple",
        in_stock=True
    )
    print(f"   ✓ Valid product created: {valid_product.name}")
    print(f"   Price: ${valid_product.price}")
    print(f"   Category: {valid_product.category}")
except Exception as e:
    print(f"   ✗ Error: {e}")

# Test 2: Invalid price
print("\n2. Testing invalid price (negative):")
try:
    invalid_product = Product(
        name="Test Product",
        price=-10.00,
        category=ProductCategory.ELECTRONICS
    )
except Exception as e:
    print(f"   ✗ Correctly caught error: {e}")

# Test 3: Missing required field
print("\n3. Testing missing required field:")
try:
    missing_field = Product(
        name="Test Product",
        price=10.00
        # Missing category!
    )
except Exception as e:
    print(f"   ✗ Correctly caught error: {e}")

# Test 4: Invalid category
print("\n4. Testing invalid category:")
try:
    wrong_category = Product(
        name="Test Product",
        price=10.00,
        category="invalid_category"  # Not in our Enum
    )
except Exception as e:
    print(f"   ✗ Correctly caught error: {e}")

print("\n✓ Product models working correctly with Pydantic V2 syntax!")

TESTING PRODUCT MODELS (Pydantic V2 Syntax)

1. Testing valid product:
   ✓ Valid product created: iPhone 15 Pro
   Price: $999.99
   Category: ProductCategory.ELECTRONICS

2. Testing invalid price (negative):
   ✗ Correctly caught error: 1 validation error for Product
price
  Input should be greater than 0 [type=greater_than, input_value=-10.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

3. Testing missing required field:
   ✗ Correctly caught error: 1 validation error for Product
category
  Field required [type=missing, input_value={'name': 'Test Product', 'price': 10.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

4. Testing invalid category:
   ✗ Correctly caught error: 1 validation error for Product
category
  Input should be 'electronics', 'clothing', 'home_goods', 'books' or 'other' [type=enum, input_value='invalid_category', input_type=str]
    For further information vis